# FastMap -- Data Mining

In this doc, we need to figure out these points:

1. How it works?
2. Why it works?
3. How to analyze its performance?
4. How to use it in application?
5. What's the comparison results with other state-out-arts?
6. What's the current evolvements of it?

## Intuition of the idea

Designing feature extraction functions can be hard. It is relatively easier for a domain expert to assess the similarity/distance of two objects. Given only the distance information though, it is not obvious how to map objects into points.

And here the goal is to embedding objects into Eclidean space as points which are $k$-dimentional vectors.

### Why choose Euclidean space?

**Enable Spatial access methods (SAMs)** to answer queries more efficiently.

Retrieval of distances: although its more clear to know the similarity of two objects, but it may be computational difficult. And here comes the benefit of embedding at Euclidean space.

Moreover, for visualization and data-mining (unsupervised).

PS: we need to think about two kinds of queries for SAM.

- `query-by-example`, whether it's easy to embedding a new node into this space.
- `all pairs query`, this requires a very fast calculation of every pair, maybe even just uses a close-formed equation.

## How it works?

Input: $\mathcal{D} (O_i,O_j)$ a distance/dis-similarity function for every pair of nodes.

Output: the embedding. $P_i=(x_{i,1},x_{i,2},...,x_{i,k})$ for $O_i$

Requirements: $\mathcal{D}()$ is:

- Non-negative
- Symmetric
- Obeys the triangular inequality (Why it matters?)

Because in the algorithm, we will use the distances' triangular inequality to form a a triangle, then use this to calculate a distance in one dimention. And as we can see, the basic of this is the triangular relationship exists.

### Distances for directed graph

**Symmetric and obeys triangular inequality**

1. $\mathcal{D} (n_i,n_j) = \frac{d_{ij}+d_{ji}}{2}$
2. $\mathcal{D} (n_i,n_j) = \max\{d_{ij}, d_{ji}\}$

**Symmetric but don't obeys triangular inequality**

1. $\mathcal{D} (n_i,n_j) = \frac{|d_{ij}-d_{ji}|}{2}$
2. $\mathcal{D} (n_i,n_j) = \min\{d_{ij}, d_{ji}\}$

## Comparison with benchmark

Two ways to evaluate it:

1. Time efficiency or even consider storage efficiency.
2. Preservation of original relations (distance) 

### Multi-Dimensional Scaling

Two metrics for such embedding:

1. Stress:
$$stress = \sqrt{\frac{\sum_{i,j}(\hat{d}_{ij}-d_{ij})^2}{\sum_{i,j}d_{ij}^2}}$$

2. NRMSD

The idea here is to use gradient decent to improve 

### Dimensionality Reduction Techniques

Cannot be applied to 'distance' case, and it's very slow on large dataset.

### Retrieval and Clustering

In a word, some methods try to use the space information to help retrieval inforamtion. Even though not all of them use Euclidean space as a "target" space, but efficient search is a common question behind them. Thus pruning is vital in this process, and triangular inequality is very useful here.

In [ ]:
from random import choice
C = 3
def fastmap(N, Distance, K, epsilon):
    embedding = {}
    obj_list = list(range(N))
    for node in obj_list:
        embedding[node] = []
    for r in range(K):
        node_a = choice(obj_list)
        node_b = node_a
        # Find the farthest nodes a, b
        for t in range(C):
            length = 0
            for j in obj_list:
                dis = Distance.getdis(node_a, j)
                if dis > length:
                    node_c = j
                    length = dis
            if node_c == node_b:
                break
            else:
                node_b = node_a
                node_a = node_c
        dis_ab = Distance.getdis(node_a, node_b)
        if dis_ab < epsilon:
            break
        # Calcute the embedding
        for node in obj_list:
            dis_na = Distance.getdis(node_a, node)
            dis_nb = Distance.getdis(node_b, node)
            p_nr = float(dis_na**2 + dis_ab**2 - dis_nb **2)/(2 * dis_ab)
            embedding[node].append(p_nr)
        
        # Update the weight of the graph
        for i in range(N):
            for j in range(i+1, N):
                Distance.changedis(i, j, embedding[i][r], embedding[j][r])
    return embedding